In [24]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, BatchNormalization, MaxPooling2D
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
input_data = np.array(np.load('./charge.npy'))
output_data = np.load('./truevals.npy')

In [4]:
print 'Shape of Input Data: {}'.format(np.shape(input_data))
print 'Shape of Output Data: {}'.format(np.shape(output_data))

Shape of Input Data: (3081, 86, 64)
Shape of Output Data: (3081, 3)


In [13]:
tvt_ratio=[3,2,2] ##ratio of test validation and test dataset
data_len = len(input_data)
test_end = int(float(tvt_ratio[0])/np.sum(tvt_ratio)*data_len)
valid_end = int(float(tvt_ratio[1])/np.sum(tvt_ratio)*data_len)+test_end
print 'Range of training dataset {}:{}'.format(0,test_end)
print 'Range of validation dataset {}:{}'.format(test_end+1,valid_end)
print 'Range of test dataset {}:{}'.format(valid_end+1,data_len)

Range of training dataset 0:1320
Range of validation dataset 1321:2200
Range of test dataset 2201:3081


In [48]:
# # ---------------------------------------------------------
# # Boilerplate. You can ignore this part.
# # ---------------------------------------------------------
# # try:
# #     CONDOR_ID = os.environ['CONDOR_ID']
# # except:
# #     sys.exit('Error: Run this script with "pygpu %file"')

# # folder = 'train-DCNN-%s' % CONDOR_ID  # folder for training results
# # os.makedirs(folder)

# # limit available GPU memory to allow for 2 jobs per GPU, please do not change!
# # config = tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.45))
# # keras.backend.tensorflow_backend.set_session(tf.Session(config=config))
# # ---------------------------------------------------------


# # split train, validation and test samples
folders=['train_hist']
for folder in folders:
    if not folder in os.listdir('.'):
        os.makedirs('./{}'.format(folder))
train = input_data[0:test_end]
valid = input_data[test_end+1:valid_end]
test  = input_data[valid_end+1:data_len-1]
train_out = np.concatenate(output_data[0:test_end,0:1])
valid_out = np.concatenate(output_data[test_end+1:valid_end, 1:2])
test_out = np.concatenate(output_data[valid_end+1:data_len-1, 2:3])


# # ----------------------------------------------------------
# # Define model
# # ----------------------------------------------------------


def add_block(nfilters, dropout=False, **kwargs):
    """ 
    Add basic convolution block: 
     - 3x3 Convolution with padding
     - Activation: ReLU
     - either MaxPooling to reduce resolution, or Dropout
     - BatchNormalization
    """
    model.add(Convolution2D(nfilters, 3, 3, border_mode='same', init="he_normal", **kwargs))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    if dropout:
        model.add(Dropout(dropout))
    else:
        model.add(MaxPooling2D((2, 2), border_mode='same'))

# # convolution part
def network():
    model = Sequential()
    add_block(16, dropout=0.3, input_shape=(86, 64,1))  # --> (32, 32, 32)
    #add_block(16)  # --> (16, 16,  32)
    model.add(Flatten()) 
    model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
#     model.add(Flatten()) 
#     model.add(Dense(1))
    #print(model.summary())
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
    return model


# # ----------------------------------------------------------
# # Training
# # ----------------------------------------------------------

estimator = KerasRegressor(build_fn=network, nb_epoch=20, batch_size=5, verbose=0)
seed = 7
np.random.seed(seed)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, train, train_out , cv=kfold)
# model.compile(
#     loss='categorical_crossentropy', 
#     optimizer=keras.optimizers.Adam(lr=1E-3),
#     metrics=['accuracy'])

# model.fit(train,
#     batch_size=32,
#     nb_epoch=25,
#     validation_data=valid,
#     callbacks=[keras.callbacks.CSVLogger(folder +'/history.csv')])

model.save('./train_hist/model.h5')  # save trained network

# print('Model performance (loss, accuracy)')
# print('Train: %.4f, %.4f' % tuple(model.evaluate(train, verbose=0, batch_size=128)))
# print('Valid: %.4f, %.4f' % tuple(model.evaluate(valid, verbose=0, batch_size=128)))
# print('Test:  %.4f, %.4f' % tuple(model.evaluate(test,  verbose=0, batch_size=128)))


# # # ----------------------------------------------------------
# # # Plots
# # # ----------------------------------------------------------
# # # training curves
# # history = np.genfromtxt(folder+'/history.csv', delimiter=',', names=True)

# # fig, ax = plt.subplots(1)
# # ax.plot(history['epoch'], history['loss'],     label='training')
# # ax.plot(history['epoch'], history['val_loss'], label='validation')
# # ax.legend()
# # ax.set(xlabel='epoch', ylabel='loss')
# # fig.savefig(folder+'/loss.png')

# # fig, ax = plt.subplots(1)
# # ax.plot(history['epoch'], history['acc'],     label='training')
# # ax.plot(history['epoch'], history['val_acc'], label='validation')
# # ax.legend()
# # ax.set(xlabel='epoch', ylabel='accuracy')
# # fig.savefig(folder+'/accuracy.png')


# # # calculate predictions for test set
# # Y_predict = model.predict(X_test, batch_size=128)

# # # convert back to class labels (0-9)
# # Y_predict_cl = np.argmax(Y_predict, axis=1)  
# # Y_test_cl    = np.argmax(Y_test, axis=1)

# # # compare prediction and truth
# # m = Y_predict_cl == Y_test_cl
# # i0 = np.arange(5000)[~m]  # misclassified images
# # i1 = np.arange(5000)[ m]  # correctly classified images

/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/IPython/kernel/__main__.py:44: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", kernel_initializer="he_normal", input_shape=(86, 64, 1...)`


ValueError: Input 0 is incompatible with layer conv2d_13: expected ndim=4, found ndim=2

In [51]:
print np.shape(train_out)
print np.shape(train)

(1320,)
(1320, 86, 64)


In [42]:
print test.ndim

3


In [ ]:

# # add_block(64, dropout=0.4)  # --> (16, 16,  64)
# # add_block(64)               # --> ( 8,  8,  64)
# # add_block(128, dropout=0.4) # --> ( 8,  8, 128)
# # add_block(128, dropout=0.4) # --> ( 8,  8, 128)
# # add_block(128)              # --> ( 4,  4, 128)
# # add_block(256, dropout=0.4) # --> ( 4,  4, 256)
# # add_block(256, dropout=0.4) # --> ( 4,  4, 256)
# # add_block(256)              # --> ( 2,  2, 256)

# # # classification part
# # model.add(Flatten())        # --> (1024)
# # model.add(Dropout(0.5))
# # model.add(Dense(256))       # --> (256)
# # model.add(BatchNormalization())
# # model.add(Activation('relu'))
# # model.add(Dropout(0.5))
# # model.add(Dense(10))        # --> (10)
# # model.add(Activation('softmax'))